In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,BatchNormalization
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, Constant, identity
from tensorflow.python.framework.ops import EagerTensor
from matplotlib.pyplot import imshow


from test_utils import summary, comparator
import public_tests

%matplotlib inline
np.random.seed(1)
tf.random.set_seed(2)

In [3]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):
        # Unpack the filters list
    F1, F2, F3 = filters
    
    # Save the input tensor for the shortcut connection
    X_shortcut = X
    
    # First CONV layer
    X = tf.keras.layers.Conv2D(F1, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X, training=training)  # Ensure training argument is passed to BatchNormalization
    X = tf.keras.layers.Activation('relu')(X)
    
    # Second CONV layer
    X = tf.keras.layers.Conv2D(F2, (f, f), strides=(1, 1), padding='same', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X, training=training)  # Ensure training argument is passed to BatchNormalization
    X = tf.keras.layers.Activation('relu')(X)
    
    # Third CONV layer
    X = tf.keras.layers.Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X, training=training)  # Ensure training argument is passed to BatchNormalization
    
    # Add the shortcut connection to the output
    X = tf.keras.layers.Add()([X_shortcut, X])
    X = tf.keras.layers.Activation('relu')(X)
    return X

In [5]:
# Testing the function with the example provided
np.random.seed(1)

# Create dummy input tensors
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

# Concatenate to form the input batch
X = np.concatenate((X1, X2, X3), axis=0).astype(np.float32)

# Run identity block with training=False
A3 = identity_block(X, f=2, filters=[4, 4, 3],
                    initializer=Constant(value=1),
                    training=False)

print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()

# Print mean of the output at specified indices
print(np.around(A3np[:,(0, -1), :, :].mean(axis=3), 5))
resume = A3np[:,(0, -1), :, :].mean(axis=3)
print(resume[1, 1, 0])

# Run identity block with training=True
print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                    initializer=Constant(value=1),
                    training=True)

print(np.around(A4.numpy()[:, (0, -1), :, :].mean(axis=3), 5))

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71236 192.71236 192.71236  96.85618]
  [ 96.85618  96.85618  96.85618  48.9281 ]]

 [[578.13715 578.13715 578.13715 290.56854]
  [290.56854 290.56854 290.56854 146.78427]]]
96.85618

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]


In [7]:
def convolutional_block(X, f, filters, s=2, training=True, initializer=tf.keras.initializers.GlorotUniform(seed=0)):
    F1, F2, F3 = filters
    X_shortcut = X

    # First CONV layer
    X = tf.keras.layers.Conv2D(F1, (1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = tf.keras.layers.Activation('relu')(X)

    # Second CONV layer
    X = tf.keras.layers.Conv2D(F2, (f, f), strides=(1, 1), padding='same', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = tf.keras.layers.Activation('relu')(X)

    # Third CONV layer
    X = tf.keras.layers.Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X, training=training)

    # Shortcut path
    X_shortcut = tf.keras.layers.Conv2D(F3, (1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer)(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)

    # Add shortcut to main path
    X = tf.keras.layers.Add()([X, X_shortcut])
    X = tf.keras.layers.Activation('relu')(X)

    return X

In [9]:
from outputs import convolutional_block_output1, convolutional_block_output2
np.random.seed(1)
initializer = tf.keras.initializers.GlorotUniform(seed=0)

# Dummy input tensors
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis=0).astype(np.float32)
A = convolutional_block(X, f=2, filters=[2, 4, 6], training=False, initializer=initializer)
assert type(A) == EagerTensor, "Use only tensorflow and keras functions"
assert tuple(tf.shape(A).numpy()) == (3, 2, 2, 6), "Wrong shape."
assert np.allclose(A.numpy(), convolutional_block_output1), "Wrong values when training=False."
print(A[0])

tf.Tensor(
[[[0.         0.66683817 0.         0.         0.888539   0.5274254 ]
  [0.         0.65053666 0.         0.         0.8959285  0.49965227]]

 [[0.         0.6312079  0.         0.         0.86362475 0.47643146]
  [0.         0.56883204 0.         0.         0.8553412  0.417093  ]]], shape=(2, 2, 6), dtype=float32)


In [11]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):

    initializer=tf.keras.initializers.GlorotUniform(seed=0)
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=initializer)(X)
    X = BatchNormalization(axis=3)(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, filters=[64, 64, 256])
    X = identity_block(X, 3, filters=[64, 64, 256])

    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, filters=[128, 128, 512])
    X = identity_block(X, 3, filters=[128, 128, 512])
    X = identity_block(X, 3, filters=[128, 128, 512])

    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, filters=[256, 256, 1024])
    X = identity_block(X, 3, filters=[256, 256, 1024])
    X = identity_block(X, 3, filters=[256, 256, 1024])
    X = identity_block(X, 3, filters=[256, 256, 1024])
    X = identity_block(X, 3, filters=[256, 256, 1024])

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, filters=[512, 512, 2048])
    X = identity_block(X, 3, filters=[512, 512, 2048])

    X = AveragePooling2D((2, 2))(X) 

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs = X_input, outputs = X)
    
    return model

In [13]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
print(model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ zero_padding2d                │ (None, 70, 70, 3)         │               0 │ input_layer[0][0]          │
│ (ZeroPadding2D)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_10 (Conv2D)            │ (None, 32, 32, 64)        │           9,472 │ zero_padding2d[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_10        │ (None, 32, 32, 64)        │             256 │ conv2d_10[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_9 (Activation)     │ (None, 32, 32, 64)        │               0 │ batch_normalization_10[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 15, 15, 64)        │               0 │ activation_9[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_11 (Conv2D)            │ (None, 15, 15, 64)        │           4,160 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_11        │ (None, 15, 15, 64)        │             256 │ conv2d_11[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_10 (Activation)    │ (None, 15, 15, 64)        │               0 │ batch_normalization_11[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_12 (Conv2D)            │ (None, 15, 15, 64)        │          36,928 │ activation_10[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_12        │ (None, 15, 15, 64)        │             256 │ conv2d_12[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_11 (Activation)    │ (None, 15, 15, 64)        │               0 │ batch_normalization_12[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_13 (Conv2D)            │ (None, 15, 15, 256)       │          16,640 │ activation_11[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_14 (Conv2D)            │ (None, 15, 15, 256)       │          16,640 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_13        │ (None, 15, 15, 256)       │           1,024 │ conv2d_13[0][0]            │
│ (BatchNormalization)          │                           │               

 Total params: 23,600,006 (90.03 MB)

 Trainable params: 23,546,886 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

None


In [14]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)
#comparator(summary(model), ResNet50_summary)

In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


In [21]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32)

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 37s 505ms/step - accuracy: 0.3043 - loss: 2.5989
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 18s 543ms/step - accuracy: 0.6789 - loss: 1.0557
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 18s 518ms/step - accuracy: 0.8636 - loss: 0.3514
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 507ms/step - accuracy: 0.9462 - loss: 0.1677
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 511ms/step - accuracy: 0.9402 - loss: 0.1669
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 513ms/step - accuracy: 0.9460 - loss: 0.2084
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 503ms/step - accuracy: 0.9573 - loss: 0.1192
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 18s 517ms/step - accuracy: 0.9612 - loss: 0.1098
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 18s 519ms/step - accuracy: 0.9865 - loss: 0.0425
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 17s 508ms/step - accuracy: 0.9814 - loss: 0.0634


In [29]:
model.save('my_model.keras')
print("Model Saved Succcessfully")

Model Saved Succcessfully


In [ ]:
model = tf.keras.models.load_model('my_model.keras')

In [31]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.3025 - loss: 2.8405
Loss = 2.866612195968628
Test Accuracy = 0.3083333373069763
